In [ ]:
%env OMP_NUM_THREADS=4
%env CUDA_VISIBLE_DEVICES=0
%load_ext autoreload
%autoreload 2

In [ ]:
### TODO: Modify to your own data dir ###
data_dir = '/data/users/pavel_i/datasets/'

In [ ]:
import sys
sys.path.append("../wilds_exps_utils/")

import torch
import numpy as np
import tqdm
import pickle
import copy
from types import SimpleNamespace
from wilds import get_dataset
from wilds.common.data_loaders import get_eval_loader
from wilds_configs import datasets as dataset_configs
from wilds.datasets.wilds_dataset import WILDSSubset
from wilds_models.initializer import initialize_model
import wilds_transforms as transforms

# from wilds_algorithms.initializer import infer_d_out

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

## Data and model

In [ ]:
DATASET = 'civilcomments'

def get_data(args, wilds_config):
    full_dataset = get_dataset(dataset=DATASET, download=False, root_dir=args.root_dir)
    transform = transforms.initialize_transform(
            transform_name=wilds_config.transform,
            config=wilds_config,
            dataset=full_dataset,
            additional_transform_name=None,
            is_training=False)

    test_data = full_dataset.get_subset("test", transform=transform)
    val_data = full_dataset.get_subset("val", transform=transform)

    if args.dfr_reweighting_drop:
        train_data = full_dataset.get_subset("train", transform=transform)
        idx = train_data.indices.copy()
        rng = np.random.default_rng(args.dfr_reweighting_seed)
        rng.shuffle(idx)
        n_train = int((1 - args.dfr_reweighting_frac) * len(idx))
        val_idx = idx[n_train:]
        reweighting_data = WILDSSubset(
                full_dataset,
                indices=val_idx,
                transform=transform)
        del train_data
    else:
        reweighting_data = val_data
    return val_data, test_data, reweighting_data

In [ ]:
# Just need the ds object for evaluation
DATASET = 'civilcomments'
seed = 2
args = SimpleNamespace(
    root_dir=data_dir,
    batch_size=16,
    dfr_reweighting_drop=True,
    dfr_reweighting_seed=seed,
    dfr_reweighting_frac=0.2,
)
wilds_config = SimpleNamespace(
    algorithm='ERM',
    load_featurizer_only=False,
    pretrained_model_path=None,
    **dataset_configs.dataset_defaults[DATASET],
)
wilds_config.model_kwargs = {}
wilds_config.model = 'bert-base-uncased'

val_data, test_data, reweighting_data = get_data(args, wilds_config)
reweighting_loader = get_eval_loader("standard", reweighting_data, batch_size=args.batch_size)
val_loader = get_eval_loader("standard", val_data, batch_size=args.batch_size)
test_loader = get_eval_loader("standard", test_data, batch_size=args.batch_size)

In [ ]:
def load_model(args, wilds_config, d_out):
    model = initialize_model(config=wilds_config, d_out=d_out)
    ckpt_dict = torch.load(args.ckpt_path)
    model.load_state_dict({k[len('model.'):]: v for (k, v) in ckpt_dict['algorithm'].items()})
    model.cuda()
    model.eval()
    classifier = model.classifier
    model.classifier = torch.nn.Identity(model.classifier.in_features)
    return model, classifier

## Extract embeddings

In [ ]:
def get_embeddings_predictions(feature_extractor, classifier, loader):
    all_embeddings, all_predictions, all_y_true, all_metadata = [], [], [], []
    with torch.no_grad():
        for x, y_true, metadata in tqdm.tqdm(loader):
            embeddings = feature_extractor(x.cuda())
            predictions = torch.argmax(classifier(embeddings), axis=1)
            all_embeddings.append(embeddings.cpu())
            all_predictions.append(predictions.cpu())
            all_y_true.append(y_true.cpu())
            all_metadata.append(metadata)
    all_embeddings = torch.cat(all_embeddings, axis=0)
    all_predictions = torch.cat(all_predictions, axis=0)
    all_y_true = torch.cat(all_y_true, axis=0)
    all_metadata = torch.cat(all_metadata, axis=0)
    return all_embeddings, all_predictions, all_y_true, all_metadata

In [ ]:
def save_emb(ckpt_path, seed, save_path):
        args = SimpleNamespace(
        root_dir=data_dir,
        batch_size=16,
        dfr_reweighting_drop=True,
        dfr_reweighting_seed=seed,
        dfr_reweighting_frac=0.2,
        ckpt_path=ckpt_path
        )
        wilds_config = SimpleNamespace(
        algorithm='ERM',
        load_featurizer_only=False,
        pretrained_model_path=None,
        **dataset_configs.dataset_defaults[DATASET],
        )
        wilds_config.model_kwargs = {}
        wilds_config.model = 'bert-base-uncased'

        val_data, test_data, reweighting_data = get_data(args, wilds_config)
        bad_test_idx = 73529 # this example is somehow broken ...
        test_data.indices = np.delete(test_data.indices, bad_test_idx)
        reweighting_loader = get_eval_loader("standard", reweighting_data, batch_size=args.batch_size)
        val_loader = get_eval_loader("standard", val_data, batch_size=args.batch_size)
        test_loader = get_eval_loader("standard", test_data, batch_size=args.batch_size)

        d_out = 2
        feature_extractor, classifier = load_model(args, wilds_config, d_out)

        reweighting_embeddings, reweighting_predictions, reweighting_y, reweighting_metadata = get_embeddings_predictions(
                feature_extractor, classifier, reweighting_loader)
        val_embeddings, val_predictions, val_y, val_metadata = get_embeddings_predictions(
                feature_extractor, classifier, val_loader)
        test_embeddings, test_predictions, test_y, test_metadata = get_embeddings_predictions(
                feature_extractor, classifier, test_loader)
        torch.save(
        dict(
                e=reweighting_embeddings, y=reweighting_y, pred=reweighting_predictions, m=reweighting_metadata,
                test_e=test_embeddings, test_pred=test_predictions, test_y=test_y, test_m=test_metadata,
                val_e=val_embeddings, val_pred=val_predictions, val_y=val_y, val_m=val_metadata,
                w0 = classifier.weight.cpu(),
                b0 = classifier.bias.cpu()
        ),
        save_path
        )

In [ ]:
from glob import glob
import os
ckpt_dir = '/data/users/pavel_i/wilds_ckpts' # TODO: change to your ckpt dir
seeds = [0, 1, 2] # TODO: change to your seeds for splitting the train data
save_dir = '../emb/civil'
os.makedirs(save_dir, exist_ok=True)
for seed in seeds:
    ckpt_path = glob(f'{ckpt_dir}/bert_civilcomments_dfrdrop_{seed}/*last_model.pth') # TODO: change to your ckpt path
    assert len(ckpt_path) == 1, f'Found {len(ckpt_path)} ckpts for seed {seed}, expected 1'
    ckpt_path = ckpt_path[0]
    save_path = f'{save_dir}/{seed}.pt'
    save_emb(ckpt_path, seed, save_path)

## AFR

In [ ]:
def plot_results(metrics):
    print("Early stopping by Val")
    earlystop_epoch = np.argmax(np.array(metrics.val_accs["wga"]))
    print(f"test WGA: {metrics.test_accs['wga'][earlystop_epoch]:.3f} at epoch {earlystop_epoch}") 
    # print(f"Early stopping test mean acc: {metrics.test_accs['mean_acc'][earlystop_epoch]:.3f} at epoch {earlystop_epoch}")
    print()
    
    print("Early stopping by Val")
    earlystop_epoch = np.argmax(metrics.val_accs["combined_wga"])
    print(f"test WGA: {metrics.test_accs['wga'][earlystop_epoch]:.3f} at epoch {earlystop_epoch}") 
    # print(f"Early stopping test mean acc: {metrics.test_accs['mean_acc'][earlystop_epoch]:.3f} at epoch {earlystop_epoch}")
    print()
    
    # 3 horizontal subplots
    fig, arr = plt.subplots(1, 3, figsize=(20, 5))
    arr[0].plot(metrics.losses, label="wxe")
    arr[0].plot(metrics.regs, label="Reg")
    arr[0].grid()
    arr[0].legend(loc="lower right")
    arr[0].set_xlabel("Epochs")
    arr[0].set_ylabel("Loss")

    arr[1].plot(metrics.test_accs["wga"], label="Test")
    arr[1].plot(metrics.reweighting_accs["wga"], label="Reweighting")
    arr[1].plot(metrics.val_accs["wga"], label="Validation")
    arr[1].grid()
    arr[1].legend(loc="lower right")
    arr[1].set_xlabel("Epochs")
    arr[1].set_ylabel("WGA")
    arr[1].set_ylim(0.5, 0.75)

    arr[2].plot(metrics.test_accs["mean_acc"], label="Test")
    arr[2].plot(metrics.reweighting_accs["mean_acc"], label="Reweighting")
    arr[2].plot(metrics.val_accs["mean_acc"], label="Validation")
    arr[2].grid()
    arr[2].legend(loc="lower right")
    arr[2].set_xlabel("Epochs")
    arr[2].set_ylabel("Mean Acc")

    # 4 horizontal subplots
    fig, arr = plt.subplots(2, 3, figsize=(20, 10))

    colors = ['r', 'g', 'b', 'y', 'm', 'c', 'k', 'orange']
    for attr, c in enumerate(colors):
        arr[0, 0].plot([ls[attr] for ls in metrics.loss_by_groups['toxic']], color=c)
        arr[0, 0].plot([ls[attr] for ls in metrics.loss_by_groups['non_toxic']], '--', color=c)
    arr[0, 0].grid()
    arr[0, 0].set_xlabel("Epochs")
    arr[0, 0].set_ylabel("Loss")
    arr[0, 0].legend(loc="lower right")

    for attr, c in enumerate(colors):
        arr[0, 1].plot([ls[attr] for ls in metrics.weights_by_groups['toxic']], color=c)
        arr[0, 1].plot([ls[attr] for ls in metrics.weights_by_groups['non_toxic']], '--', color=c)
    arr[0, 1].grid()
    arr[0, 1].set_xlabel("Epochs")
    arr[0, 1].set_ylabel("Weights")
    arr[0, 1].legend(loc="lower right")

    for attr, c in enumerate(colors):
        arr[0, 2].plot([ls[attr] for ls in metrics.confidence_by_groups['toxic']], color=c)
        arr[0, 2].plot([ls[attr] for ls in metrics.confidence_by_groups['non_toxic']], '--', color=c)
    arr[0, 2].grid()
    arr[0, 2].set_xlabel("Epochs")
    arr[0, 2].set_ylabel("Confidence")
    arr[0, 2].legend(loc="lower right")

    for attr, c in enumerate(colors):
        arr[1, 0].plot([ls[attr] for ls in metrics.test_accs['by_group_toxic']], color=c)
        arr[1, 0].plot([ls[attr] for ls in metrics.test_accs['by_group_nontoxic']], '--', color=c)
    arr[1, 0].plot(metrics.test_accs['combined_wga'], '-*', color='k')
    arr[1, 0].grid()
    arr[1, 0].set_xlabel("Epochs")
    arr[1, 0].set_ylabel("Test Acc")
    arr[1, 0].legend(loc="lower right")
    
    for attr, c in enumerate(colors):
        arr[1, 1].plot([ls[attr] for ls in metrics.val_accs['by_group_toxic']], color=c)
        arr[1, 1].plot([ls[attr] for ls in metrics.val_accs['by_group_nontoxic']], '--', color=c)
    arr[1, 1].plot(metrics.val_accs['combined_wga'], '-*', color='k')
    arr[1, 1].grid()
    arr[1, 1].set_xlabel("Epochs")
    arr[1, 1].set_ylabel("Val Acc")
    arr[1, 1].legend(loc="lower right")
    
    for attr, c in enumerate(colors):
        arr[1, 2].plot([ls[attr] for ls in metrics.reweighting_accs['by_group_toxic']], color=c)
        arr[1, 2].plot([ls[attr] for ls in metrics.reweighting_accs['by_group_nontoxic']], '--', color=c)
    arr[1, 2].plot(metrics.reweighting_accs['combined_wga'], '-*', color='k')
    arr[1, 2].grid()
    arr[1, 2].set_xlabel("Epochs")
    arr[1, 2].set_ylabel("Reweighting Acc")
    arr[1, 2].legend(loc="lower right")

In [ ]:
class Metrics():
    def __init__(
            self, reweighting_metadata, test_metadata, val_metadata):
        
        self.n_cols = 8
        
        self.reweighting_metadata = reweighting_metadata
        self.test_metadata = test_metadata
        self.val_metadata = val_metadata
        
        self.reweighting_accs = self.get_init_accs()
        self.test_accs = self.get_init_accs()
        self.val_accs = self.get_init_accs()
        
        self.weights_by_groups = self.get_init_dict()
        self.loss_by_groups = self.get_init_dict()
        self.confidence_by_groups = self.get_init_dict()
        
        keys = ("toxic", "non_toxic")
        self.reweighting_group_sizes = dict(zip(keys, self._something_by_groups(
                torch.ones_like(reweighting_metadata[:, 0]), reweighting_metadata)))
        self.val_group_sizes = dict(zip(keys, self._something_by_groups(
                torch.ones_like(val_metadata[:, 0]), val_metadata)))
        self.test_group_sizes = dict(zip(keys, self._something_by_groups(
                torch.ones_like(test_metadata[:, 0]), test_metadata)))
        
        self.losses = []
        self.regs = []
        
#         self.weighted_acc = []
    
    @staticmethod
    def get_init_dict():
        return {'toxic': [], 'non_toxic': []}
    
    @staticmethod
    def get_init_accs():
        return {
            'mean_acc': [],
            'wga': [],
            'combined_wga': [],
            'by_group_toxic': [],
            'by_group_nontoxic': []
        }
    
    def _something_by_groups(self, something, metadata, mean=False):
        if mean: 
            return (
                [something[(metadata[:, i] == 1) & (metadata[:, -2] == v)].float().mean().item() for i in range(self.n_cols)]
                for v in [0, 1])
        else:
            return (
                [something[(metadata[:, i] == 1) & (metadata[:, -2] == v)].sum().item() for i in range(self.n_cols)]
                for v in [0, 1])
    
    def _update_weights_by_groups(self, weights):
        toxic, nontoxic = self._something_by_groups(
            weights, self.reweighting_metadata)
        self.weights_by_groups['toxic'].append(toxic)
        self.weights_by_groups['non_toxic'].append(nontoxic)
    
    def _update_loss_by_groups(self, ce):
        toxic, nontoxic = self._something_by_groups(
            ce, self.reweighting_metadata)
        self.loss_by_groups['toxic'].append(toxic)
        self.loss_by_groups['non_toxic'].append(nontoxic)
    
    def _update_confidence_by_groups(self, p_true):
        toxic, nontoxic = self._something_by_groups(
            p_true, self.reweighting_metadata, mean=True)
        self.confidence_by_groups['toxic'].append(toxic)
        self.confidence_by_groups['non_toxic'].append(nontoxic)
    
    def _update_accs(self, accs, metadata, results, predictions):
        toxic, nontoxic = self._something_by_groups(predictions == metadata[:, -2], metadata, mean=True)
        accs['mean_acc'].append(results[0]['acc_avg'])
        accs['wga'].append(results[0]['acc_wg'])
        accs['by_group_toxic'].append(toxic)
        accs['by_group_nontoxic'].append(nontoxic)
        all_accs = np.array([toxic, nontoxic])
        weights = np.array([0.56, 0.29, 0.15])[None, :]
        all_accs = np.hstack([
            all_accs[:, :3], 
            np.sum(all_accs[:, 3:6] * weights , axis=1, keepdims=True),
            all_accs[:, 6:]])
        accs['combined_wga'].append(np.min(all_accs))

   
    def update(
            self, weights, ce, p_true, loss, reg,
            reweighting_results, reweighting_predictions,
            test_results, test_predictions, 
            val_results, val_predictions
        ):
        
        self._update_weights_by_groups(weights)
        self._update_loss_by_groups(ce)
        self._update_confidence_by_groups(p_true)
        
        self._update_accs(self.reweighting_accs, self.reweighting_metadata, reweighting_results, reweighting_predictions)
        self._update_accs(self.test_accs, self.test_metadata, test_results, test_predictions)
        self._update_accs(self.val_accs, self.val_metadata, val_results, val_predictions)
        
        self.losses.append(loss.item())
        self.regs.append(reg.item())

    

In [ ]:
def get_wxe(*_):
    def confidence_wxe_fn(logits, y, weights):
        ce = torch.nn.functional.cross_entropy(logits, y, reduction='none')
        loss = weights * ce
        return loss.sum(), weights

    return confidence_wxe_fn

def compute_weights(logits, reweighting_y, gamma, balance_classes, base_weights=0.):
    with torch.no_grad():
        p = logits.softmax(-1)
        y_onehot = torch.zeros_like(logits).scatter_(-1, reweighting_y.unsqueeze(-1), 1)
        p_true = (p * y_onehot).sum(-1)
        weights = (-gamma * p_true).exp()
        if balance_classes:
            w1 = (reweighting_y == 0).sum()
            w2 = (reweighting_y == 1).sum()
            weights[reweighting_y == 0] *= w2 / w1
        weights = weights.detach()
        weights /= weights.sum()
        weights += base_weights / len(weights)
    return weights

In [ ]:
def train(
        reweighting_data, test_data, val_data,
        emb_dict, num_epochs, gamma, reg_coeff, lr=1e-2,
        base_weights=0.,
        balance_classes=False, plot=True, silent=False, combine_reweighting=False):

    del plot
    
    reweighting_embeddings = emb_dict['e'].cuda()
    reweighting_y = emb_dict['y'].cuda()
    reweighting_metadata = emb_dict['m'].cuda()
    test_embeddings = emb_dict['test_e'].cuda()
    test_y = emb_dict['test_y'].cuda()
    test_metadata = emb_dict['test_m'].cuda()
    val_embeddings = emb_dict['val_e'].cuda()
    val_y = emb_dict['val_y'].cuda()
    val_metadata = emb_dict['val_m'].cuda()
    w0 = emb_dict['w0'].cuda()
    b0 = emb_dict['b0'].cuda()
    
    if combine_reweighting:
        reweighting_embeddings = torch.cat([reweighting_embeddings, val_embeddings])
        reweighting_y = torch.cat([reweighting_y, val_y])
        reweighting_metadata = torch.cat([reweighting_metadata, val_metadata])
    print("Reweighting size:", len(reweighting_y))
    
    # Early stopping
    best_weights = {'w': w0, 'b': b0}
    best_wga = 0

    last_layer_model = torch.nn.Linear(w0.shape[1], w0.shape[0], bias=True)
    last_layer_model.weight.data = w0.clone()
    last_layer_model.bias.data = b0.clone()
    last_layer_model.cuda()

    criterion = get_wxe(gamma)
    optimizer = torch.optim.SGD(last_layer_model.parameters(), lr=lr, weight_decay=0., momentum=0.)

    metrics = Metrics(reweighting_metadata, test_metadata, val_metadata)

    initial_logits = last_layer_model(reweighting_embeddings)
    weights = compute_weights(initial_logits, reweighting_y, gamma, balance_classes, base_weights=base_weights)

    for epoch in (pbar := tqdm.tqdm(range(num_epochs), disable=silent)):
        optimizer.zero_grad()
        logits = last_layer_model(reweighting_embeddings)
        loss, _ = criterion(logits, reweighting_y, weights)

        # L2 penalty for deviation from model.fc.weight and model.fc.bias
        reg = ((last_layer_model.weight - w0).pow(2).sum() + (last_layer_model.bias - b0).pow(2).sum())
        loss += reg_coeff * reg

        # additional infos
        with torch.no_grad():
            ce = torch.nn.functional.cross_entropy(logits, reweighting_y, reduction='none')
            p_true = (logits.softmax(-1) * torch.nn.functional.one_hot(reweighting_y, 2)).sum(-1)
            
            reweighting_predictions = torch.argmax(logits, -1)
            test_predictions = torch.argmax(last_layer_model(test_embeddings), -1)
            val_predictions = torch.argmax(last_layer_model(val_embeddings), -1)
            reweighting_results = reweighting_data.eval(
                    reweighting_predictions.cpu(), reweighting_y.cpu(), reweighting_metadata.cpu())
            test_results = test_data.eval(test_predictions.cpu(), test_y.cpu(), test_metadata.cpu())
            val_results = val_data.eval(val_predictions.cpu(), val_y.cpu(), val_metadata.cpu())
        
        metrics.update(
            weights, ce, p_true, loss, reg,
            reweighting_results, reweighting_predictions,
            test_results, test_predictions, 
            val_results, val_predictions
        )
        
        wga = metrics.val_accs['combined_wga'][-1]
        if wga > best_wga:
            best_wga = wga
            best_weights = {'w': last_layer_model.weight.data.clone(), 'b': last_layer_model.bias.data.clone()}

        nzw = (weights != 0).float().mean()
        loss.backward()
        # clip gradients
        torch.nn.utils.clip_grad_norm_(last_layer_model.parameters(), 1.)
        optimizer.step()
    #     scheduler.step()

        wga = metrics.reweighting_accs["wga"][-1]
        test_wga = metrics.test_accs["wga"][-1]
        pbar.set_description(f"Val cwga: {best_wga:.3f}, TWGA: {test_wga:.3f}")

    return metrics, best_weights, best_wga

In [ ]:
def group_size_adjusted_wga(accs, sizes, i):
    at, st = np.array(accs["by_group_toxic"][i]), np.array(sizes["toxic"])
    an, sn = np.array(accs["by_group_nontoxic"][i]), np.array(sizes["non_toxic"])
    vt, vn = at - at**2, an - an**2
    lbs = np.concatenate([at - np.sqrt(vt / st), an - np.sqrt(vn / sn)])
    return np.min(lbs)

def wga(accs, i):
    at = np.array(accs["by_group_toxic"][i])
    an = np.array(accs["by_group_nontoxic"][i])
    return np.min(np.concatenate([at, an]))

## Hyper tuning

In [ ]:
def twga(metrics):
    earlystop_epoch = np.argmax(metrics.val_accs["combined_wga"])
    return metrics.test_accs['wga'][earlystop_epoch]

def twga_last(metrics):
    earlystop_epoch = -1
    return metrics.test_accs['wga'][earlystop_epoch]

def vwga(metrics):
    earlystop_epoch = np.argmax(metrics.val_accs["wga"])
    return metrics.val_accs['wga'][earlystop_epoch]

def vwga_last(metrics):
    earlystop_epoch = -1
    return metrics.val_accs['wga'][earlystop_epoch]

def vcwga(metrics):
    earlystop_epoch = np.argmax(metrics.val_accs["combined_wga"])
    return metrics.val_accs['combined_wga'][earlystop_epoch]

In [ ]:
gammas = [0., 0.001, 0.01, 0.1, 1., 10., 100.]
reg_coefs = [0]
lrs = [1e-2]
vps = [1.] # validation proportion: change / add more if you want to subsample the validation set

all_results = {}
all_metrics = {}

for val_prop in vps:
    for seed in seeds:
        for shuffle_seed in [0, 21, 42]:
            rng = np.random.default_rng(shuffle_seed)
            emb_dict = torch.load(f"{save_dir}/{seed}.pt")
            idx = np.arange(len(emb_dict['val_y']))
            rng.shuffle(idx)
            idx = idx[:int(val_prop * len(idx))]
            emb_dict['val_y'] = emb_dict['val_y'][idx]
            emb_dict['val_e'] = emb_dict['val_e'][idx]
            emb_dict['val_m'] = emb_dict['val_m'][idx]
            for gamma in gammas:
                for reg_coef in reg_coefs:
                    for lr in lrs:
                        metrics, weights, val_wga = train(
                                reweighting_data, test_data, val_data,
                                emb_dict, num_epochs=50, gamma=gamma, reg_coeff=reg_coef, lr=lr,
                                balance_classes=True, plot=False, silent=False, combine_reweighting=False)
                        all_results[(val_prop, shuffle_seed, seed, gamma, reg_coef, lr)] = val_wga
                        all_metrics[(val_prop, shuffle_seed, seed, gamma, reg_coef, lr)] = metrics
                        print(f"seed {seed}, gamma {gamma}, reg {reg_coef}, lr {lr}: {val_wga}")
# save all_metrics
torch.save(all_metrics, "all_metrics.pt")

In [ ]:
# select best hyperparameters
all_metrics = torch.load("all_metrics.pt")
seeds = [0,1,2]
test_wgas = []
for val_prop in vps:
    twgas = []
    for seed in seeds:
        seed_metrics = {k: v for (k, v) in all_metrics.items() if k[0] == val_prop and k[2] == seed}
        max_key = max(seed_metrics, key=lambda k: vcwga(seed_metrics[k]))
        twgas.append(twga(all_metrics[max_key]))
    test_wgas.append(twgas)
    print(val_prop, np.mean(twgas), np.std(twgas))

mean_wgas = [np.mean(twgas) for twgas in test_wgas]
std_wgas = [np.std(twgas) for twgas in test_wgas]
mean_wgas = np.array(mean_wgas) * 100
std_wgas = np.array(std_wgas) * 100
vps = np.array(vps)

print(mean_wgas)